In [ ]:
from pathlib import Path
from TELF.pipeline.blocks import DataBundle, DIR_LIST_BUNDLE_KEY, SAVE_DIR_BUNDLE_KEY, SOURCE_DIR_BUNDLE_KEY, RESULTS_DEFAULT
from TELF.pipeline.blocks import AnimalBlock
from TELF.pipeline import DirectoryLoopBlock
from TELF.pipeline import BlockManager

In [ ]:
class ReadDirBlock(AnimalBlock):
    def __init__(self, tag="block1", needs = (SOURCE_DIR_BUNDLE_KEY,), provides = ('dirname',)):
        super().__init__(tag=tag, needs = needs, provides = provides)
    def run(self, b: DataBundle):
        b[f"{self.tag}.{self.provides[0]}"] = b[SOURCE_DIR_BUNDLE_KEY].name

class CountNameBlock(AnimalBlock):
    def __init__(self, tag="block2", needs = ("dirname",), provides = ('name_length',)):
        super().__init__(tag=tag, needs = needs, provides = provides)
    def run(self, b: DataBundle):
        b[f"{self.tag}.{self.provides[0]}"] =  len(b[self.needs[0]])

class SquareCountBlock(AnimalBlock):
    def __init__(self, tag="block3", needs = ("name_length",), provides = ('length_squared',)):
        super().__init__(tag=tag, needs = needs, provides = provides)
    def run(self, b: DataBundle):
        b[f"{self.tag}.{self.provides[0]}"] =  b[self.needs[0]] ** 2

In [ ]:
per_dir_blocks = [
    ReadDirBlock(),
    CountNameBlock(),
    SquareCountBlock(),
]
dir_loop_block =  DirectoryLoopBlock(
    subblocks=per_dir_blocks,
    max_workers=4,
    capture_output=None
)
pipeline = [
    dir_loop_block,
]

In [ ]:
SquareCountBlock().needs

In [ ]:
initial = DataBundle({
    DIR_LIST_BUNDLE_KEY: [
        "/path/to/folderA/with/some/random/files",
        "/path/to/folderB/short_path",
        "/path/to/folderC/another/longer/path/with_more_subdirs_and_files",
    ],
    SAVE_DIR_BUNDLE_KEY: Path(".")
})

In [ ]:
mgr = BlockManager(
    pipeline, 
    initial,
    capture_output=None
)
final_bundle = mgr()

In [ ]:
final_bundle.keys()

In [ ]:
for result in final_bundle[RESULTS_DEFAULT]:
    print(result.get('dir'))
    print(result.get('dirname'))
    print(result.get('name_length'))
    print(result.get('length_squared'))